<a href="https://colab.research.google.com/github/issmythe/intepretable-ai-crops/blob/main/dl_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#@title Mount drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Imports
import sys
sys.path.append('drive/MyDrive/current_research_projects/utils/')
sys.path.append('drive/MyDrive/current_research_projects/dl_yield_forecasts/code/src/py/')

import copy
import importlib
import itertools
import math
import os
import pickle
import random
import time

import numpy as np
import pandas as pd

# Analysis
from datetime import datetime, timedelta
from scipy.stats import pearsonr

from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

# Plotting
import matplotlib
import plotly
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly import subplots

# Utils
from read_data import get_max_vi, get_modis_vi, get_sif, get_weather, get_yields
from dl_helpers import fit_predict_fold, test
from data_object import DataObj, DataObj2D, DataObjPretrain, BaggingDataObj, DummyDataObj

from networks.network import Network
from networks.basic_cnn import BasicCNN
from networks.basic_lstm import BasicLSTM
from networks.hybrid_lstm import HybridLSTM
from networks.segmented_cnn import SegmentedCNN

# Tensorflow
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

%load_ext tensorboard


In [ ]:
#@title Constants
DATA_PATH = 'drive/MyDrive/current_research_projects/us_data/'
DL_DATA_PATH = 'drive/MyDrive/current_research_projects/dl_yield_forecasts/data/'

# HEAT_DATA_PATH = 'drive/MyDrive/research_ideas/heat_separability/data/'
datestr = datetime.today().strftime('%Y%m')
FIG_PATH = f'drive/MyDrive/current_research_projects/dl_yield_forecasts/figs/{datestr}/'
! mkdir -p $FIG_PATH

PROCESS_DATA = False

# Data

In [ ]:
PROCESS_DATA = False
END_YEAR_INC = 2021

In [ ]:
#@title Get yields
yields = get_yields(2000, END_YEAR_INC, True).drop('Unnamed: 0', axis=1)
# yields.to_csv(DL_DATA_PATH + 'preprocessed_data/yields_east_2000_2021.csv', index=False)

yield_pre_preds = pd.read_csv(f'{DL_DATA_PATH}/preprocessed_data/pretrain_labels.csv')
yield_pre_preds['label_pred'] = yield_pre_preds.apply(
    lambda x: [x[f'pred_{i + 1}'] for i in range(17)], axis=1)
yield_pre_preds['label_delta'] = yield_pre_preds.apply(
    lambda x: [x[f'delta_{i}'] for i in range(17)], axis=1)
yield_pre_preds = yield_pre_preds[['fips', 'year', 'label_pred', 'label_delta']]
yield_pre_preds['fips'] = yield_pre_preds['fips'].apply(lambda x: str(x).zfill(5))


<ipython-input-5-993951d2b92b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yield_pre_preds['fips'] = yield_pre_preds['fips'].apply(lambda x: str(x).zfill(5))


In [ ]:
#@title Get ddays
if False:
    full_ddays = pd.read_csv(
        f'{DATA_PATH}/weather/full_ddays_full_year/all_years_with_temp20240315.csv')

    full_ddays = full_ddays.merge(
        yields[['year']].assign(fips=yields['fips'].apply(lambda x: int(x))))

    # Format data
    full_ddays = full_ddays.sort_values('doy')
    full_ddays['dday10C'] -= full_ddays['dday29C']
    full_ddays = full_ddays[full_ddays['doy'] < 365]

    # Make full-season dataframes
    dday_cols = ['dday29C', 'dday10C', 'prec']
    march_aug = full_ddays[(full_ddays['doy'] >= pd.to_datetime(f'2023-03-01').dayofyear) &
                        (full_ddays['doy'] < pd.to_datetime(f'2023-09-01').dayofyear)]\
        .groupby(['fips', 'year'])[dday_cols].sum().reset_index()

    april_sept = full_ddays[(full_ddays['doy'] >= pd.to_datetime(f'2023-04-01').dayofyear) &
                            (full_ddays['doy'] < pd.to_datetime(f'2023-10-01').dayofyear)]\
        .groupby(['fips', 'year'])[dday_cols].sum().reset_index()

    # Pivot and merge
    weather_cols = dday_cols + ['tMin', 'tMax']
    full_ddays = full_ddays[['fips', 'year', 'doy'] + weather_cols]
    full_ddays = full_ddays.groupby(['fips', 'year']).agg({c: list for c in weather_cols}).reset_index()

    daily_data = full_ddays.merge(
        march_aug.rename({c: f'{c}_march_aug' for c in weather_cols}, axis=1)).merge(
        april_sept.rename({c: f'{c}_april_sept' for c in weather_cols}, axis=1))

    daily_data['fips'] = daily_data['fips'].apply(lambda x: str(x).zfill(5))


In [ ]:
#@title Get EVI
if False:
    evi = pd.read_csv(DATA_PATH + 'evi/mean_cdl_2000_2021.csv')
    evi['fips'] = evi['GEOID'].apply(lambda x: str(x).zfill(5))

    evi['date'] = pd.to_datetime(evi['date'])
    evi['year'] = evi['date'].dt.year
    evi['doy'] = evi['date'].dt.dayofyear
    evi = evi.merge(yields[['fips', 'year']])[['fips', 'year', 'date', 'doy', 'evi']]

    start, end = 60, 365 - 31
    all_dates = evi[['fips', 'year']].drop_duplicates().merge(pd.DataFrame(
        {'doy': range(1, end)}), how='cross')
    evi_all = evi.merge(all_dates, how='right')
    interp = evi_all.groupby(['fips', 'year'])\
        .apply(lambda group: group['evi'].interpolate(method='linear'))
    interp = interp.reset_index().drop('level_2', axis=1)
    interp['doy'] = [x for x in range(1, end)] * len(interp[['fips', 'year']].drop_duplicates())
    evi_all = evi_all.merge(interp.rename({'evi': 'evi_interp'}, axis=1))

    evi_mini = evi_all[(evi_all['doy'] > start) & (evi_all['doy'] < end)]
    evi_mini['idx'] = evi_all['doy'].apply(lambda x: (x - start) % 15)
    evi_mini = evi_mini[evi_mini['idx'] == 0].groupby(['fips', 'year'])['evi_interp'].apply(list)\
        .reset_index().rename({'evi_interp': 'evi_mini'}, axis=1)
    evi_all = evi_all[evi_all['doy'] >= start].groupby(['fips', 'year'])['evi_interp'].apply(list)\
        .reset_index().rename({'evi_interp': 'evi'}, axis=1)

    daily_data = evi_all.merge(evi_mini).merge(daily_data)
    daily_data.to_csv(
        DL_DATA_PATH + 'preprocessed_data/prediction_df_through_2021_with_temp.csv', index=False)

In [ ]:
#@title Read pre-processed features
daily_data = pd.read_csv(
    DL_DATA_PATH + 'preprocessed_data/prediction_df_through_2021_with_temp.csv')
daily_data['fips'] = daily_data['fips'].apply(lambda x: str(x).zfill(5))

to_list = lambda x: [float(v) for v in x.strip('[').strip(']').split(', ')]
for col in ['evi', 'evi_mini', 'dday10C', 'dday29C', 'prec', 'tMin', 'tMax']:
    daily_data[col] = daily_data[col].apply(to_list)

weather_cols = ['dday29C', 'dday10C', 'prec']
daily_data = daily_data.rename({f'{c}_march_aug': f'{c}_total' for c in weather_cols}, axis=1)\
    .drop([f'{c}_april_sept' for c in weather_cols], axis=1)

daily_data = daily_data.merge(yields).assign(reg_year=daily_data['year'] - daily_data['year'].min())
daily_data = pd.get_dummies(daily_data, columns=['fips', 'state']).assign(
    fips=daily_data['fips'], state=daily_data['state'])


In [ ]:
#@title Filter and format pre-processed features
def clip(df, var, thresh=0.0005):
    return df[(df[var] >= df[var].quantile(thresh)) & (df[var] <= df[var].quantile(1 - thresh))]

clipped_dd_29 = clip(daily_data, 'dday29C_total')
clipped_dd_10 = clip(daily_data, 'dday10C_total')
clipped_dd_prec = clip(daily_data, 'prec_total')

feature_df = daily_data.merge(clipped_dd_29[['fips', 'year']])\
    .merge(clipped_dd_10[['fips', 'year']])\
    .merge(clipped_dd_prec[['fips', 'year']])

for col in ['dday10C', 'dday29C', 'prec', 'tMin', 'tMax']:
    feature_df[col] = feature_df[col].apply(lambda x: x[60:-30])
    feature_df[f'{col}_total'] = feature_df[col].apply(sum)


In [ ]:
#@title Get EVI pre-train labels
evi_ts_df = pd.read_csv(f'{DL_DATA_PATH}/preprocessed_data/pretrain_evi.csv')
evi_ts_df = evi_ts_df.groupby(['fips', 'year'])['evi_z'].apply(list).reset_index().merge(
    evi_ts_df.groupby(['fips', 'year'])['evi_z_delta'].apply(list).reset_index())
evi_ts_df['fips'] = evi_ts_df['fips'].apply(lambda x: str(x).zfill(5))

feature_df = feature_df.merge(evi_ts_df)


In [ ]:
#@title Constants - toggle between temp and ddays
ts_cols = ['tMin', 'tMax', 'prec']

state_cols = [x for x in feature_df.columns if x.startswith('state_')]
fips_cols = [x for x in feature_df.columns if x.startswith('fips_')]
year_cols = [
    x for x in feature_df.columns if x.startswith('reg_year_') or x.startswith('reg_year2_')]
flat_cols = ['reg_year'] + fips_cols

random.seed(123)
years = [x for x in range(2000, END_YEAR_INC + 1)]
random.shuffle(years)
FOLDS = np.array_split(years, 10)

np.random.seed(123)
train_years_gs = [x for x in range(2000, 2017)]
np.random.shuffle(train_years_gs)
GS_FOLDS = np.array_split(train_years_gs, 5)


In [ ]:
#@title Eval helpers
def get_rmse(c1, c2):
    return ((c1 - c2) ** 2).mean() ** 0.5

def get_corr(c1, c2):
    return pearsonr(c1, c2)[0]

def get_r2(c1, c2):
    return pearsonr(c1, c2)[0] ** 2

def get_lm_performance(fold, train=False):
    def _get_stats(df):
        if train:
            sample_df = df[~df['year'].isin(FOLDS[fold])]
        else:
            sample_df = df[df['year'].isin(FOLDS[fold])]
        return round(get_rmse(sample_df['pred'], sample_df['log_yield']), 3), \
               round(pearsonr(sample_df['pred'], sample_df['log_yield'])[0], 3)

    print('Baseline RMSE, R:', _get_stats(lm_preds_bl))
    print('March RMSE, R:', _get_stats(lm_preds_march))
    print('April RMSE, R:', _get_stats(lm_preds_april))
    print('USDA season RMSE, R:', _get_stats(lm_preds_season))


In [ ]:
# Shared grid search helper
def get_results_df(param_dict, all_preds, best_iters):
    results_df = pd.DataFrame(param_dict, index=[0])
    results_df['rmse'] = get_rmse(all_preds['log_yield'], all_preds['pred'])
    results_df['corr'] = get_corr(all_preds['log_yield'], all_preds['pred'])
    results_df['best_iters'] = [best_iters]
    return results_df

In [ ]:
#@title Read linear model results
HEAT_DATA_PATH = 'drive/MyDrive/current_research_projects/heat_separability/data/'
yield_lm_dir = f'{HEAT_DATA_PATH}/yield_predictions/'

lm_preds_bl = pd.read_csv(f'{yield_lm_dir}/baseline_preds_31_states.csv')
lm_preds_march = pd.read_csv(f'{yield_lm_dir}/march_through_aug_31_states.csv')
lm_preds_april = pd.read_csv(f'{yield_lm_dir}/april_through_sept_31_states.csv')
lm_preds_season = pd.read_csv(f'{yield_lm_dir}/doy_full_season_31_states.csv')

def quick_summarize(preds):
    rmse = get_rmse(preds['log_yield'], preds['pred'])
    r2 = get_r2(preds['log_yield'], preds['pred'])
    print('N =', len(preds), 'RMSE:', round(rmse, 3), 'R2:', round(r2, 3))

print('Expected N:', len(yields))
quick_summarize(lm_preds_bl)
quick_summarize(lm_preds_april)
quick_summarize(lm_preds_march)
quick_summarize(lm_preds_season)

for df in [lm_preds_bl, lm_preds_april, lm_preds_march, lm_preds_season]:
    train_df = df[df['year'] < 2017]
    print(round(get_rmse(train_df['log_yield'], train_df['pred']), 3),
          round(get_corr(train_df['log_yield'], train_df['pred']), 3))

Expected N: 33064
N = 33064 RMSE: 0.24 R2: 0.503
N = 33064 RMSE: 0.208 R2: 0.626
N = 33064 RMSE: 0.203 R2: 0.644
N = 33064 RMSE: 0.201 R2: 0.65
0.257 0.664
0.219 0.769
0.213 0.783
0.212 0.786


In [ ]:
! mv $DL_DATA_PATH2/preprocessed_data/pretrain_evi.csv $DL_DATA_PATH/preprocessed_data/

mv: cannot stat '/preprocessed_data/pretrain_evi.csv': No such file or directory


# EVI exploratory analysis

In [ ]:
#@title Helper to calculate z-scores
def get_z_score_data(df, col):
    mean_fn = lambda g: np.mean(np.stack(g[col].values), axis=0)
    std_fn = lambda g: np.std(np.stack(g[col].values), axis=0)

    z_data = df[df['year'] <= 2016]
    mean_ts = z_data.groupby(['fips']).apply(mean_fn).reset_index()
    std_ts = z_data.groupby(['fips']).apply(std_fn).reset_index()
    state_mean_ts = z_data.groupby(['state']).apply(mean_fn).reset_index()
    state_std_ts = z_data.groupby(['state']).apply(std_fn).reset_index()

    zdf = df[['fips', 'state', 'year', col]].merge(
        mean_ts.rename({0: f'county_mean_{col}'}, axis=1), how='left').merge(
        std_ts.rename({0: f'county_std_{col}'}, axis=1), how='left').merge(
        state_mean_ts.rename({0: f'state_mean_{col}'}, axis=1), how='left').merge(
        state_std_ts.rename({0: f'state_std_{col}'}, axis=1), how='left')

    zdf[f'county_mean_{col}'] = zdf[f'county_mean_{col}'].fillna(zdf[f'state_mean_{col}'])
    zdf[f'county_std_{col}'] = zdf[f'state_std_{col}'].fillna(zdf[f'state_mean_{col}'])
    zdf[f'{col}_z'] = np.divide(
        np.subtract(zdf[f'{col}'], zdf[f'county_mean_{col}']), zdf[f'county_std_{col}'])\
        .apply(lambda x: np.nan_to_num(x, posinf=0))
    return zdf[['fips', 'state', 'year', col, f'{col}_z']]


In [ ]:
#@title Get EVI df
n_periods, n_days, shift_days = 17, 17, 16

# Get z-score data
evi_ts_df = get_z_score_data(feature_df, 'evi')

# Explode df to get EVI variables for end of each period
def _index_evi(i):
    return evi_ts_df[['fips', 'state', 'year']].assign(
        evi=evi_ts_df['evi'].apply(lambda x: x[shift_days * i + n_days]),
        evi_z=evi_ts_df['evi_z'].apply(lambda x: x[shift_days * i + n_days]),
        idx=i)

start_evi = evi_ts_df[['fips', 'state', 'year']].assign(
    evi=evi_ts_df['evi'].apply(lambda x: x[0]),
    evi_z=evi_ts_df['evi_z'].apply(lambda x: x[0]),
    idx=-1)
evi_df = pd.concat([start_evi] + [_index_evi(i) for i in range(n_periods)])

# Add delta variables and yield data
evi_df['evi_prev'] = evi_df.groupby(['fips', 'year'])['evi'].shift(1)
evi_df['evi_z_prev'] = evi_df.groupby(['fips', 'year'])['evi_z'].shift(1)
evi_df['evi_delta'] = evi_df['evi'] - evi_df['evi_prev']
evi_df['evi_z_delta'] = evi_df['evi_z'] - evi_df['evi_z_prev']
evi_df = evi_df[evi_df['idx'] >= 0].merge(yields)

# Get correlation for each period
evi_corr_df = evi_df.groupby('idx')\
    [['log_yield', 'evi', 'evi_z', 'evi_delta', 'evi_z_delta']].corr().reset_index()
evi_corr_df = evi_corr_df[evi_corr_df['level_1'] == 'log_yield'].drop(
    ['level_1', 'log_yield'], axis=1)


In [ ]:
#@title Plot EVI vs yield over time
fig = plotly.subplots.make_subplots(rows=2, cols=1)

fig.add_trace(go.Scatter(name='EVI', x=evi_corr_df['idx'], y=evi_corr_df['evi']), row=1, col=1)
fig.add_trace(go.Scatter(name='∆EVI', x=evi_corr_df['idx'], y=evi_corr_df['evi_delta']))
fig.add_trace(go.Scatter(name='EVI z-score', x=evi_corr_df['idx'], y=evi_corr_df['evi_z']))
fig.add_trace(go.Scatter(name='∆EVI z-score', x=evi_corr_df['idx'], y=evi_corr_df['evi_z_delta']))

fig.update_yaxes(title='Correlation with log yield', row=1, col=1)
fig.update_xaxes(title='Time step', row=1, col=1)

mean_evi_ts = evi_df.groupby('idx')['evi'].mean().reset_index()

fig.add_trace(go.Scatter(name='EVI', x=mean_evi_ts['idx'], y=mean_evi_ts['evi']), row=2, col=1)

fig.update_layout(height=800, width=800)
# plotly.io.write_image(fig, f'{FIG_PATH}/evi_vs_yield.png', scale=2)
fig.show()


In [ ]:
#@title Get baseline yield predictions
state_cols = [x for x in feature_df.columns if x.startswith('state_')]

for c in state_cols:
    feature_df[f'year_{c}'] = feature_df[c] * feature_df['reg_year']
    feature_df[f'year_{c}_2'] = feature_df[c] * feature_df['reg_year'] ** 2

year_cols = [x for x in feature_df.columns if x.startswith('year_state_')]

train_df = feature_df[feature_df['year'] <= 2016]
train_fips = train_df[flat_cols].sum().reset_index()
train_fips = train_fips[train_fips[0] > 0]['index'].to_list()

baseline_model = linear_model.LinearRegression().fit(
    y=train_df['log_yield'], X=train_df[train_fips + year_cols])


In [ ]:
#@title EVI vs. yield over time
baseline_preds = train_df[['fips', 'year', 'log_yield']].assign(
    pred_0=baseline_model.predict(train_df[train_fips + year_cols]))

evi_var = 'evi_z'
for i in range(17):
    df_i = baseline_preds.merge(evi_df[evi_df['idx'] == i])
    model_i = linear_model.LinearRegression().fit(y=df_i['log_yield'], X=df_i[[evi_var, f'pred_{i}']])
    baseline_preds = baseline_preds.assign(
        **{f'pred_{i + 1}': model_i.predict(df_i[[evi_var, f'pred_{i}']])})
    rmse = get_rmse(baseline_preds['log_yield'], baseline_preds[f'pred_{i + 1}'])
    corr = get_corr(baseline_preds['log_yield'], baseline_preds[f'pred_{i + 1}'])

    print(i, round(rmse, 3), round(corr, 3))



In [ ]:
#@title EVI vs. yield residual over time

def get_rmse_corr(evi_var, verbose):
    baseline_preds = feature_df[['fips', 'year', 'log_yield']].assign(
        pred_0=baseline_model.predict(feature_df[train_fips + year_cols]))
    baseline_preds['residual'] = baseline_preds['log_yield'] - baseline_preds['pred_0']
    baseline_preds['pred_0'] = 0

    corrs, rmses = [], []
    for i in range(17):
        df_i = baseline_preds.merge(evi_df[evi_df['idx'] == i])
        train_i = df_i[df_i['year'] <= 2016]

        model_i = linear_model.LinearRegression().fit(
            y=train_i['residual'], X=train_i[[evi_var, f'pred_{i}']])
        baseline_preds = baseline_preds.assign(
            **{f'pred_{i + 1}': model_i.predict(df_i[[evi_var, f'pred_{i}']])})

        eval_preds = baseline_preds[baseline_preds['year'] <= 2016]
        rmse = get_rmse(eval_preds['residual'], eval_preds[f'pred_{i + 1}'])
        corr = get_corr(eval_preds['residual'], eval_preds[f'pred_{i + 1}'])
        rmses.append(rmse)
        corrs.append(corr)
        if verbose:
            print(i, round(rmse, 3), round(corr, 3))

    return rmses, corrs, baseline_preds

rmse_evi_z, corr_evi_z, yield_pre_preds = get_rmse_corr('evi_z', False)
rmse_evi, corr_evi, _ = get_rmse_corr('evi', False)
# yield_pre_preds.to_csv(f'{DL_DATA_PATH}/preprocessed_data/pretrain_labels.csv', index=False)

In [ ]:
#@title Plot EVI vs yield over time
fig = plotly.subplots.make_subplots(rows=1, cols=2)

x = [x for x in range(17)]
fig.add_trace(go.Scatter(name='EVI', x=x, y=rmse_evi, line_color='red'), row=1, col=1)
fig.add_trace(go.Scatter(name='EVI z-score', x=x, y=rmse_evi_z, line_color='blue'), row=1, col=1)

fig.add_trace(go.Scatter(showlegend=False, x=x, y=corr_evi, line_color='red'), row=1, col=2)
fig.add_trace(go.Scatter(showlegend=False, x=x, y=corr_evi_z, line_color='blue'), row=1, col=2)


fig.update_yaxes(title='RMSE vs. yield residual', row=1, col=2)
fig.update_yaxes(title='Correlation vs. yield residual', row=1, col=2)
fig.update_xaxes(title='Time step')


fig.update_layout(height=600, width=1000)
# plotly.io.write_image(fig, f'{FIG_PATH}/evi_vars_vs_yield_residual.png', scale=2)
fig.show()


In [ ]:
# @title Make weather df
# Get z scores for weather vars
prec_df = get_z_score_data(feature_df, 'prec')
tmin_df = get_z_score_data(feature_df, 'tMin')
tmax_df = get_z_score_data(feature_df, 'tMax')
weather_z = prec_df.merge(tmin_df).merge(tmax_df)
eval_df = weather_z[weather_z['year'] <= 2016]

# Split by time period
def index_array(x, idx):
    return x[shift_days * idx: shift_days * idx + n_days]

eval_df = pd.concat([
    eval_df.assign(**{c: eval_df[c].apply(lambda x: index_array(x, i)) for c in ts_cols})
    .assign(**{f'{c}_z': eval_df[f'{c}_z'].apply(lambda x: index_array(x, i)) for c in ts_cols})
    .assign(idx=i) for i in range(n_periods)])

# Get average weather var values by time period
for col in ts_cols:
    eval_df[f'{col}_avg'] = eval_df[col].apply(np.mean)
    eval_df[f'{col}_z_avg'] = eval_df[f'{col}_z'].apply(np.mean)


# Add yield data
def get_yield_labels(i):
    return yield_pre_preds[['fips', 'year', f'pred_{i + 1}', f'delta_{i}']].rename(
        {f'pred_{i + 1}': f'pred', f'delta_{i}': 'delta' }, axis=1).assign(idx=i)

for i in range(17):
    yield_pre_preds[f'delta_{i}'] = yield_pre_preds[f'pred_{i + 1}'] - yield_pre_preds[f'pred_{i}']

eval_df = eval_df.drop(ts_cols + [f'{c}_z' for c in ts_cols], axis=1).merge(
    pd.concat([get_yield_labels(i) for i in range(17)]))


In [ ]:
#@title Predict pre-train variables with average weather by period

def _fit_predict(col_fn, y_var, i, df=None):
    if i is not None:
        pdf = eval_df[eval_df['idx'] == i]
    else:
        pdf = eval_df

    train = pdf[~pdf['year'].isin(FOLDS[0])]
    test = pdf[pdf['year'].isin(FOLDS[0])]

    preds = linear_model.LinearRegression().fit(
        X=train[[col_fn(c) for c in ts_cols]], y=train[y_var]).predict(
        test[[col_fn(c) for c in ts_cols]])
    return get_rmse(preds, test[y_var]), get_corr(preds, test[y_var])

result_dfs = []

for i in range(17):
    rmse_pred, r_pred = _fit_predict(lambda c: f'{c}_avg', 'pred', i)
    rmse_pred_z, r_pred_z = _fit_predict(lambda c: f'{c}_z_avg', 'pred', i)
    rmse_delta, r_delta = _fit_predict(lambda c: f'{c}_avg', 'delta', i)
    rmse_delta_z, r_delta_z = _fit_predict(lambda c: f'{c}_z_avg', 'delta', i)

    i_df = dict(idx=i, rmse_pred=rmse_pred, r_pred=r_pred,
                rmse_pred_z=rmse_pred_z, r_pred_z=r_pred_z,
                rmse_delta=rmse_delta, r_delta=r_delta,
                rmse_delta_z=rmse_delta_z, r_delta_z=r_delta_z)
    result_dfs.append(pd.DataFrame(i_df, index=[0]))

weather_to_yield = pd.concat(result_dfs)


In [ ]:
#@title Predict pre-train variables with average weather, pooled
_, r_pred = _fit_predict(lambda c: f'{c}_avg', 'pred', i=None)
_, r_pred_z = _fit_predict(lambda c: f'{c}_z_avg', 'pred', i=None)
_, r_delta = _fit_predict(lambda c: f'{c}_avg', 'delta', i=None)
_, r_delta_z = _fit_predict(lambda c: f'{c}_z_avg', 'delta', i=None)
print(round(r_pred, 3), round(r_pred_z, 3), round(r_delta, 3), round(r_delta_z, 3))

In [ ]:
#@title Plot average weather vs. pre-train variables
fig = plotly.subplots.make_subplots()

x = weather_to_yield['idx']

fig.add_trace(go.Scatter(name='Weather vs. pred', x=x, y=weather_to_yield['r_pred']))
fig.add_trace(go.Scatter(name='Weather z-score vs. pred', x=x, y=weather_to_yield['r_pred_z']))
fig.add_trace(go.Scatter(name='Weather vs. delta', x=x, y=weather_to_yield['r_delta']))
fig.add_trace(go.Scatter(name='Weather z-score vs. delta', x=x, y=weather_to_yield['r_delta_z']))

fig.update_yaxes(title='Correlation')
fig.update_xaxes(title='Time step')


fig.update_layout(height=600, width=1000)
# plotly.io.write_image(fig, f'{FIG_PATH}/weather_vs_pretraining_labels.png', scale=2)
fig.show()


# Grid search

In [ ]:
#@title Grid search helpers
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()

def do_gs_cv(param_dict, ts_cols, flat_cols, init_model_fn):
    all_preds, best_iters = [], []
    t0 = time.time()
    for i in range(len(GS_FOLDS)):
        model = init_model_fn(use_best_params=False, kwargs=param_dict)
        model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=param_dict['lr'])

        do = DataObj(feature_df, GS_FOLDS[i], GS_FOLDS[i], train_years_gs, ts_cols, flat_cols)
        print(do.train_pd['year'].unique(), do.test_pd['year'].unique(), do.val_pd['year'].unique())

        preds, best_iter = fit_predict_fold(do, ts_cols, flat_cols, model, optimizer,
            param_dict['n_epochs'], device, criterion, verbose=False)
        all_preds.append(preds), best_iters.append(best_iter[0])
        print(f'CV fold {i + 1} / {len(GS_FOLDS)} complete, {round(time.time() - t0)} elapsed')

    return get_results_df(param_dict, pd.concat(all_preds), best_iters)


def make_param_combos(param_grid):
    keys, values = zip(*param_grid.items())
    param_combos = [dict(zip(keys, v)) for v in itertools.product(*values)]

    np.random.seed(123)
    param_combos = np.random.choice(param_combos, len(param_combos), replace=False)
    return [x for x in param_combos if not (x['k'] == 9 and x['stride'] == 8)]


def do_grid_search(network, use_ddays, param_grid=None, start=0, end=100):
    dummy_network = network(True)
    data_str = 'ddays' if use_ddays else 'temperature'
    out_dir = f'{DL_DATA_PATH}/grid_search/{dummy_network.network_name}_{data_str}'
    ! mkdir $out_dir

    param_grid = param_grid if param_grid else dummy_network.get_param_grid()
    param_combos = make_param_combos(param_grid)
    ts_cols = ['dday10C', 'dday29C', 'prec'] if use_ddays else ['tMin', 'tMax', 'prec']
    flat_cols = ['reg_year'] + fips_cols

    print(ts_cols, set(x.split('_')[0] for x in flat_cols))
    date_str = datetime.today().strftime('%Y%m%d')

    results = []
    t0 = time.time()

    for i in range(start, end):
        iter_results = do_gs_cv(param_combos[i], ts_cols, flat_cols, network).assign(param_combo=i)
        iter_results.to_csv(f'{out_dir}/{date_str}_{i}.csv', index=False)
        print(iter_results)
        print(f'Param combo {i} complete, {round((time.time() - t0) / 60, 1)} elapsed')

    pd.concat(results).to_csv(f'{out_dir}/{date_str}_{i}.csv', index=False)


In [ ]:
#@title Do grid search
from networks.basic_cnn import BasicCNN
do_grid_search(BasicCNN, use_ddays=False, end=1)


# Bagging

In [ ]:
#@title Helper
N_BAGGING_FOLDS = 100

def do_bagging(network, use_ddays, df=feature_df, ts_cols=None, start=0, end=100):
    if ts_cols is None:
        ts_cols = ['dday10C', 'dday29C', 'prec'] if use_ddays else ['tMin', 'tMax', 'prec']
    data_str = 'dday' if use_ddays else 'temperature'
    name = network(use_best_params=True, use_ddays=use_ddays).network_name
    out_dir = f'{DL_DATA_PATH}/bagging/{name}_{data_str}'
    print('*', out_dir)
    ! mkdir $out_dir
    test_years = [x for x in range(2017, END_YEAR_INC + 1)]

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    criterion = nn.MSELoss()

    t0 = time.time()
    for i in range(start, end):
        print(i)
        model = network(use_best_params=True, use_ddays=use_ddays, read_params=False)
        if i == 0:
            print(model.best_kwargs)
        model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=model.get_best_param('lr'))
        do = BaggingDataObj(df, test_years, i + 1, ts_cols, flat_cols)

        preds, _ = fit_predict_fold(do, ts_cols, flat_cols, model, optimizer,
                                    model.get_best_param('n_epochs'), device,
                                    criterion, verbose=False)
        # preds.to_csv(f'{out_dir}/{i}.csv', index=False)

        r = get_corr(preds['pred'], preds['log_yield'])
        rmse = get_rmse(preds['pred'], preds['log_yield'])
        print(f'Bagging fold {i + 1}/{N_BAGGING_FOLDS} complete, {round(time.time() - t0)} elapsed',
              round(r, 3), round(rmse, 3))


In [ ]:
#@title Do bagging
# Framework for models other than HybridLSTM
do_bagging(BasicCNN, use_ddays=False, start=78, end=79)

# Framework for HybridLSTM
from networks.hybrid_lstm import HybridLSTM

pretrain_loc = 2
def init_hlstm(use_best_params, kwargs=None, use_ddays=True, read_params=False,
               pretrain_loc=pretrain_loc, verbose=False):
    return HybridLSTM(use_best_params, kwargs=kwargs, use_ddays=use_ddays, read_params=read_params,
                 pretrain_loc=pretrain_loc, pretrain=False, verbose=verbose)

do_bagging(init_hlstm, False, start=2)


# Pretrain workspace

In [ ]:
#@title Helpers
y_col = 'evi_z'

def set_gradients(model, freeze_layers, unfreeze):
    for name, param in model.named_parameters():
            if name in freeze_layers:
                param.requires_grad = unfreeze
            else:
                break

def get_freeze_layers(pretrain_loc):
    freeze_layers = ['conv2d.weight', 'conv2d.bias']
    if pretrain_loc == 1:
        freeze_layers += ['fc_pretrain.weight', 'fc_pretrain.bias']
    if pretrain_loc >= 2:
        freeze_layers += [
            'lstm.weight_ih_l0', 'lstm.weight_hh_l0', 'lstm.bias_ih_l0', 'lstm.bias_hh_l0']
    if pretrain_loc == 3:
        freeze_layers += ['fc1.weight', 'fc1.bias']
    return freeze_layers


def fit_predict_pretrain_fold(pretrain_do, full_do, pretrain_loc, epochs, lrs, device,
                              criterion, verbose=False, return_model=False):
    print(ts_cols)
    model = HybridLSTM(use_best_params=True, use_ddays=False, read_params=False,
                       verbose=False, pretrain_loc=pretrain_loc, pretrain=True)
    model.to(device)
    freeze_layers = get_freeze_layers(pretrain_loc)
    n_epochs_pt1, n_epochs_pt2, n_epochs_combined = epochs
    lr1, lr2, lr_combo = lrs

    # Train first half
    optimizer = optim.Adam(model.parameters(), lr=lr1)
    preds, _ = fit_predict_fold(pretrain_do, ts_cols, flat_cols, model, optimizer, n_epochs_pt1,
                                device, criterion, verbose=verbose, print_train=False)

    # Freeze first half + train second half
    set_gradients(model, freeze_layers, False)
    model.pretrain = False
    optimizer = optim.Adam(model.parameters(), lr=lr2)
    preds, _ = fit_predict_fold(full_do, ts_cols, flat_cols, model, optimizer, n_epochs_pt2, device,
                                criterion, verbose=verbose, print_train=False)

    # Train jointly & predict
    set_gradients(model, freeze_layers, True)
    optimizer = optim.Adam(model.parameters(), lr=lr_combo)
    preds, _ = fit_predict_fold(full_do, ts_cols, flat_cols, model, optimizer, n_epochs_combined,
                                device, criterion, verbose=verbose, print_train=False)
    if return_model:
        return preds, model
    else:
        return preds


## **Train sample network

In [ ]:
#@title Setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()

pretrain_loc = 3
freeze_layers = get_freeze_layers(pretrain_loc)

n_epochs_pt1, n_epochs_pt2, n_epochs_combined = 20, 10, 10
lr1, lr2, lr_combo = 1e-4, 1e-4, 5e-5

pretrain_do = DataObj(feature_df, FOLDS[0], FOLDS[0], None, ts_cols, flat_cols, y=y_col)
full_do = DataObj(feature_df, FOLDS[0], FOLDS[0], None, ts_cols, flat_cols)

In [ ]:
#@title Train
verbose = True
model = HybridLSTM(use_best_params=True, use_ddays=False, read_params=False,
                    verbose=False, pretrain_loc=pretrain_loc, pretrain=True)
model.to(device)

# Train first half
optimizer = optim.Adam(model.parameters(), lr=lr1)
preds, _ = fit_predict_fold(pretrain_do, ts_cols, flat_cols, model, optimizer, n_epochs_pt1, device,
                    criterion, verbose=verbose, print_train=False)

# Freeze first half + train second half
set_gradients(model, freeze_layers, False)
model.pretrain = False
optimizer = optim.Adam(model.parameters(), lr=lr2)
preds, _ = fit_predict_fold(full_do, ts_cols, flat_cols, model, optimizer, n_epochs_pt2, device,
                    criterion, verbose=verbose, print_train=False)

# Train jointly & predict
set_gradients(model, freeze_layers, True)
optimizer = optim.Adam(model.parameters(), lr=lr_combo)
preds, _ = fit_predict_fold(full_do, ts_cols, flat_cols, model, optimizer, n_epochs_combined,
                            device, criterion, verbose=verbose, print_train=False)


##** Grid search

In [ ]:
#@title Helpers
param_grid = dict(n_epochs_pt1=[10, 20, 35, 50],
                  n_epochs_pt2=[10, 20, 35, 50],
                  n_epochs_combined=[5, 10, 20, 35],
                  lr1=[5e-5, 7.5e-5, 1e-4],
                  lr2=[5e-5, 7.5e-5, 1e-4],
                  lr_combo=[2.5e-5, 5e-5, 7.5e-5])

def make_param_combos_pt(param_grid):
    keys, values = zip(*param_grid.items())
    param_combos = [dict(zip(keys, v)) for v in itertools.product(*values)]
    np.random.seed(123)
    pcs = np.random.choice(param_combos, len(param_combos), replace=False)
    pcs = [x for x in pcs if x['lr_combo'] < min(x['lr1'], x['lr2'])]
    return [x for x in pcs if x['n_epochs_combined'] <= min(x['n_epochs_pt1'], x['n_epochs_pt2'])]


def extract_params(param_combos, fold):
    p = param_combos[fold]
    epochs = (p['n_epochs_pt1'], p['n_epochs_pt2'], p['n_epochs_combined'])
    lr = (p['lr1'], p['lr2'], p['lr_combo'])
    return epochs, lr


def do_gs_cv_pt(param_combos, fold, ts_cols, flat_cols, pretrain_loc, device, criterion):
    epochs, lrs = extract_params(param_combos, fold)

    all_preds = []
    t0 = time.time()
    for i in range(len(GS_FOLDS)):
        pretrain_do = DataObj(
            feature_df, GS_FOLDS[i], GS_FOLDS[i], train_years_gs, ts_cols, flat_cols, y=y_col)
        full_do = DataObj(feature_df, GS_FOLDS[i], GS_FOLDS[i], train_years_gs, ts_cols, flat_cols)

        preds = fit_predict_pretrain_fold(
            pretrain_do, full_do, pretrain_loc, epochs, lrs, device, criterion, verbose=False)
        all_preds.append(preds)

        print(f'CV fold {i + 1} / {len(GS_FOLDS)} complete, {round(time.time() - t0)} elapsed')

    return get_results_df(param_combos[fold], pd.concat(all_preds))


def do_grid_search(pretrain_loc, param_grid, start=0, end=100):
    out_dir = f'{DL_DATA_PATH}/grid_search/pretrain_{pretrain_loc}_temperature'
    print('*', out_dir)
    ! mkdir $out_dir

    param_combos = make_param_combos_pt(param_grid)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    criterion = nn.MSELoss()

    t0 = time.time()
    for fold in range(start, end):
        # pre-train is set up to only run with tmin, tmax, prec as ts_cols
        iter_results = do_gs_cv_pt(
            param_combos, fold, ts_cols, flat_cols, pretrain_loc, device, criterion)
        # iter_results.assign(param_combo=fold).to_csv(f'{out_dir}/{fold}.csv', index=False)
        print(iter_results)
        print(f'Param combo {fold} complete, {round((time.time() - t0) / 60, 1)} elapsed')


In [ ]:
#@title Do grid search
do_grid_search(3, param_grid, end=1)


##**Bagging

In [ ]:
#@title Helper
N_BAGGING_FOLDS = 100

def do_pretrain_bagging(pretrain_loc, epochs, lrs, start=0, end=100):
    out_dir = f'{DL_DATA_PATH}/bagging/pretrain_{pretrain_loc}_temperature2'
    print('*****', out_dir)
    ! mkdir $out_dir
    test_years = [x for x in range(2017, END_YEAR_INC + 1)]

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    criterion = nn.MSELoss()

    t0 = time.time()
    for i in range(start, end):
        pretrain_do = BaggingDataObj(feature_df, test_years, i + 1, ts_cols, flat_cols, y='evi_z')
        full_do = BaggingDataObj(feature_df, test_years, i + 1, ts_cols, flat_cols)
        preds = fit_predict_pretrain_fold(
            pretrain_do, full_do, pretrain_loc, epochs, lrs, device, criterion, verbose=False)
        preds.to_csv(f'{out_dir}/{i}.csv', index=False)

        r = get_corr(preds['pred'], preds['log_yield'])
        rmse = get_rmse(preds['pred'], preds['log_yield'])
        print(f'Bagging fold {i + 1}/{N_BAGGING_FOLDS} complete, {round(time.time() - t0)} elapsed',
                round(r, 3), round(rmse, 3))


In [ ]:
#@title Do bagging
dir = f'{DL_DATA_PATH}/grid_search/pretrain_3_temperature.csv'
params = pd.read_csv(dir).sort_values('rmse').iloc[0]

epochs = (int(params['n_epochs_pt1']),
          int(params['n_epochs_pt2']),
          int(params['n_epochs_combined']))
lrs = (params['lr1'], params['lr2'], params['lr_combo'])

do_pretrain_bagging(3, epochs, lrs, start=1)

##**Fit in-sample models

In [ ]:
#@title Setup
years = [x for x in range(2000, END_YEAR_INC + 1)]
pretrain_do = DataObj(feature_df, [2000], [2000], years, ts_cols, flat_cols, y=y_col)
full_do = DataObj(feature_df, [2000], [2000], years, ts_cols, flat_cols)


In [ ]:
#@title Fit models
pretrain_loc = 3
dir = f'{DL_DATA_PATH}/grid_search/pretrain_{pretrain_loc}_temperature.csv'
out_dir = f'{DL_DATA_PATH}/models/pretrain_3_temperature_unnorm/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()

params = pd.read_csv(dir).sort_values('rmse').iloc[0]
epochs = (int(params['n_epochs_pt1']),
          int(params['n_epochs_pt2']),
          int(params['n_epochs_combined']))
lrs = (params['lr1'], params['lr2'], params['lr_combo'])

t0 = time.time()
for i in [0] + [x for x in range(2, 10)]:
    _, model = fit_predict_pretrain_fold(pretrain_do, full_do, pretrain_loc, epochs, lrs, device,
                                         criterion, verbose=False, return_model=True)
    torch.save(model.state_dict(), f'{out_dir}/{i}_unnorm.pth')
    print(i, '/ 10 complete,', round(time.time() - t0), 'elapsed')


In [ ]:
#@title Load models and predict
m = HybridLSTM(use_best_params=True, use_ddays=False, read_params=False,
               verbose=False, pretrain_loc=pretrain_loc, pretrain=False)
m.load_state_dict(torch.load(f'{dir}/{i}.pth'))

preds = test(m, device, full_do.train_x, full_do.train_y, criterion, predict=True)
pred_df = full_do.train_pd[['year', 'fips', 'log_yield']].assign(
    pred=full_do.ss_y.inverse_transform(preds.reshape(full_do.train_y.shape)).flatten())
get_rmse(pred_df['log_yield'], pred_df['pred'])
